In [ ]:
from keras.models import Sequential
from keras import models
from keras import layers
from keras.layers import Convolution2D,Conv2D,Activation
from keras.layers import MaxPooling2D,GlobalAveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from keras.applications.densenet import DenseNet121
from keras import optimizers
from keras import regularizers

RuntimeError: module compiled against API version 0xb but this version of numpy is 0xa

Using TensorFlow backend.


RuntimeError: module compiled against API version 0xb but this version of numpy is 0xa

ImportError: numpy.core.multiarray failed to import

ImportError: numpy.core.umath failed to import

ImportError: numpy.core.umath failed to import

In [ ]:
conv_base=DenseNet121(weights='imagenet',include_top=False)
conv_base.trainable = True

model = models.Sequential()
model.add(conv_base)
model.add(layers.GlobalAveragePooling2D())
model.add(BatchNormalization())
model.add(layers.Dense(6, activation='softmax'))
model.summary()

In [ ]:


from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   fill_mode='nearest')

train_data_dir = "training_data"
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    shuffle=True,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical')


validation_datagen = ImageDataGenerator(rescale=1. / 255)
validation_data_dir = "validation_data"
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical')


In [ ]:
from keras import optimizers
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=0.001, momentum=0.9, decay=0.005),
              metrics=['acc'])

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=50,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=1)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

x=0
y=0
for i in val_acc[-(len(epochs)//5):]:
    x+=1
    y+=i
print(y/x)

In [ ]:
model.save("dense.h5")

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import shutil
import os
os.makedirs("testing")
shutil.move("test_images","testing/test_images")
test_set = "testing"
test_datagen = ImageDataGenerator(
    rescale=1./ 255)

test_generator = test_datagen.flow_from_directory(
    directory=test_set,
    target_size=(64,64),
    batch_size=1,
    class_mode='categorical',
    shuffle=False
)

filenames = test_generator.filenames
nb_samples = len(filenames)


In [ ]:
from keras.models import load_model
model=load_model("dense.h5")
model.summary()

In [ ]:
predict = model.predict_generator(test_generator,steps = nb_samples, verbose=1)

In [ ]:
import numpy as np
print(predict[0])
label = np.where(predict[0]==max(predict[0]))

In [ ]:
label_map = train_generator.class_indices
print(label_map)

In [ ]:
csv_file = open("dense.csv","w")
csv_file.write("ID,Label\n")
for filename, prediction in zip(filenames,predict):
    name = filename.split("\\")
    print(name)
    name = name[1]
    label = np.where(prediction==max(prediction))
    label = label[0][0]
    csv_file.write(str(name)+","+str(label)+"\n")
csv_file.close()